### 아이디어
- 각 문자열의 위치의 타일이 한번이라도 교체되면 된다.
- 문자열이 매칭됐을때, 그곳의 위치와 매칭된 문자열의 길이를 같이 가져올 수 있으면 구현할 수 있을 것 같다.
- (챗봇) trie입력시 매칭하는 문자열의 길이 정보가 있다면 구할 수 있다. 단, 매칭가능한 문자가 여러개 있을 수 있다. 
  - 사실이라면, 어짜피 가장 긴 문자열이 이전 타일들을 전부 교체할 것이므로 여러개를 다 저장할 필요는 없을 것이다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**5)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output += v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        end = start + self.length[u.output]
        for i in range(start, end):
          self.vis[i] = True
        cnt += 1
    return cnt

def sol() :
  N = int(input())
  trie = Trie()
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  length = [0] 
  vis = [False] * N
  for i, s in enumerate(P) :
    trie.add(s, i+1)
    length.append(len(s)) #각 패턴의 길이를 저장
  
  aho = AhoCorasick(trie, length, vis)
  aho.query(S)

  print(aho.vis.count(False))

sol()

- 대충 정답을 출력하는 것 같긴 한데, 메모리 문제가 생겼다.
  - 트라이로 삼을 문자열의 개수와 길이가 각각 5000이라 그런것 같다..

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**4)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = 0

  def add(self, k, n) :
    if not k :
      self.output = n
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:], n) 

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie, length: list, vis: list):
    self.trie = trie
    self.trie.fail = self.trie
    self.length = length
    self.vis = vis
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output += v.fail.output
        Q.append(v)
      
  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for idx, c in enumerate(s) :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        start = idx - self.length[u.output] + 1
        end = start + self.length[u.output]
        assert end <= len(self.vis)
        for i in range(start, end):
          self.vis[i] = True
        cnt += 1
    return cnt

def sol() :
  N = int(input())
  S = input().rstrip()

  M = int(input())
  P = [input().rstrip() for _ in range(M)]
  vis = [False] * N
  
  idx = 0
  while idx < M  : #패턴을 한번에 처리할 필요가 없으므로 100개마다 쿼리를 처리한다
    trie = Trie()
    length = [0]
    for i, s in enumerate(P[idx:]) :
      if i == 100: 
        idx += 100
        break
      trie.add(s, i+1)
      length.append(len(s)) #각 패턴의 길이를 저장
    else :
      aho = AhoCorasick(trie, length, vis)
      aho.query(S)
      break
    
    aho = AhoCorasick(trie, length, vis)
    aho.query(S)

  print(aho.vis.count(False))

sol()

- 쿼리를 100개씩 처리하는 아이디어. 하지만 IndexError가 어딘가에서 발생하고 있다.